In [1]:
import pandas as pd

from fuzzywuzzy import fuzz

c:\Users\Matias\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Comenzaremos buscando duplicados entre los datasets de google  yelp.

In [2]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'metadata_FL_Google.json'
# Lee el archivo JSON y carga los datos de google en un DataFrame
df_metadada_google = pd.read_json(ruta_archivo, encoding='utf-8')


# Especifica la ruta del archivo JSON
ruta_archivo = 'yelp_business_fl.json'
# Lee el archivo JSON y carga los datos de yelp en un DataFrame
df_metadada_yelp = pd.read_json(ruta_archivo, encoding='utf-8')

In [3]:
# Eliminamos las filas que se encuentren repetidas
df_metadada_google = df_metadada_google.drop_duplicates(subset=["gmap_id"])
# Eliminamos las columnas que no tendremos en cuenta para el analisis de los duplicados
df_metadada_google = df_metadada_google.drop(['category', 'avg_rating', 'num_of_reviews', 'MISC', 'state'], axis=1)
# Ordenamos las columnas para que sea mas facil analizar las coincidencias
df_metadada_google = df_metadada_google[['gmap_id', 'name', 'latitude', 'longitude', 'address', 'estado']]
# Transformamos los datos de las columnas para geolocalizacion en numeros.
df_metadada_google['latitude'] = pd.to_numeric(df_metadada_google['latitude'], errors='coerce')
df_metadada_google['longitude'] = pd.to_numeric(df_metadada_google['longitude'], errors='coerce')

# Eliminamos las filas que se encuentren repetidas
df_metadada_yelp = df_metadada_yelp.drop_duplicates(subset=["business_id"])
# Eliminamos las columnas que no tendremos en cuenta para el analisis de los duplicados
df_metadada_yelp = df_metadada_yelp.drop(['stars', 'review_count', 'attributes', 'categories'], axis=1)
# Ordenamos las columnas para que sea mas facil analizar las coincidencias
df_metadada_yelp  = df_metadada_yelp [['business_id', 'name', 'latitude', 'longitude', 'address', 'city', 'state']]
# Transformamos los datos de las columnas para geolocalizacion en numeros.
df_metadada_yelp['latitude'] = pd.to_numeric(df_metadada_yelp['latitude'], errors='coerce')
df_metadada_yelp['longitude'] = pd.to_numeric(df_metadada_yelp['longitude'], errors='coerce')

In [4]:
df_metadada_google.head() # 15198 filas

,gmap_id,name,latitude,longitude,address,estado
0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,26.641377,-81.940545,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",FL
1,0x88c2e4e34f1ed783:0x76c5da381c499d79,Fresh Point Country Buffet,27.867489,-82.702971,"Fresh Point Country Buffet, 10525 US-19, Pinel...",FL
2,0x88dae191ee505917:0x6ba3e25388d3fad4,Oneyda's Bakery,26.154754,-81.790528,"Oneyda's Bakery, 600 Goodlette-Frank Rd #101, ...",FL
3,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,Annie's Bake Shoppe,25.717416,-80.361945,"Annie's Bake Shoppe, 10331 SW 54th St, Miami, ...",FL
4,0x88d9b99475d9fd7b:0xea6083d207b2471a,Tropical Park Liquors,25.733914,-80.325216,"Tropical Park Liquors, 7971 SW 40th St Suite #...",FL


In [5]:
df_metadada_yelp.head() # 11413 filas

,business_id,name,latitude,longitude,address,city,state
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,2575 E Bay Dr,Largo,FL
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,53.549633,-113.508780,10588 109 Street,Edmonton,FL
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,39.906295,-86.047463,6880 E 82nd St,Indianapolis,FL
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,39.869911,-86.143577,838 Broad Ripple Ave,Indianapolis,FL
4,ppFCk9aQkM338Rgwpl2F5A,Wawa,39.954573,-75.194894,3604 Chestnut St,Philadelphia,FL


In [6]:
from math import radians, sin, cos, sqrt, atan2

# Crear un DataFrame para almacenar los resultados
duplicados_lista = []

# Establecer un rango máximo de distancia para considerar como "cercanos"
rango_maximo = 0.250  # Rango en kilómetros

# Iterar sobre cada comercio en google y compararlo con cada comercio en yelp
for index1, row1 in df_metadada_google.iterrows():
    for index2, row2 in df_metadada_yelp.iterrows():
        lat1, lon1 = row1['latitude'], row1['longitude']
        lat2, lon2 = row2['latitude'], row2['longitude']

        # Calcular la distancia entre los dos comercios
        R = 6371  # Radio aproximado de la Tierra en kilómetros
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distancia = R * c

        # Si la distancia está dentro del rango establecido, considerarlos como cercanos
        if distancia <= rango_maximo:
            ratio_similitud = fuzz.ratio(row1['name'], row2['name'])
            if ratio_similitud > 80:  # Umbral de similitud entre cadenas
                # Agregar resultados al DataFrame de resultados
                duplicados_lista.append({
                    'gmap_id': row1['gmap_id'],
                    'business_id': row2['business_id'],
                    'name_g': row1['name'],
                    'name_y': row2['name'],
                    'address_g': row1['address'],
                    'address_y': f"{row2['address']}, {row2['city']}"
                })

In [7]:
# Guardamos los duplicados en un dataframe
df_duplicados = pd.DataFrame(duplicados_lista)

In [8]:
df_duplicados           # 175 filas

,gmap_id,business_id,name_g,name_y,address_g,address_y
0,0x88c2f730debd585f:0xeb36db3a693c980f,iuBcrPk2AsXu7x98tO63cA,Mainstay Tavern,Mainstay Tavern,"Mainstay Tavern, Clearwater, FL 33767","1160 Gulf Blvd, Clearwater"
1,0x88c2c4f199418f8f:0x610540c2747e67a6,NYjJ9KZM4xxcYtDpjkTSkg,Bennigan's,Bennigan's,"Bennigan's, 615 Channelside Dr #5942, Tampa, F...","615 Channelside Dr, Suite 205, Tampa"
2,0x88c32349f7b7e54b:0xd8683bee55d578e,R_PDoxc8hKzl_5TVittHHA,Publix Liquors at Gateway Commons,Publix at Gateway Commons,"Publix Liquors at Gateway Commons, 9518 Buffal...","9520 Buffalo Rd, Palmetto"
3,0x88c2fd6652a25e5b:0x25effa6bd3446b62,BSJE3vxVO9Q3SQygTEmZtQ,Marker 14,Marker 14,"Marker 14, 400 Treasure Island Causeway, Treas...","400 Treasure Island Cswy, Treasure Island"
4,0x88c2c023ffffffff:0xb846ce4c5a3a97eb,5Vrbqr9DK1gkrHGAS7y5eg,Teavana,Teavana,"Teavana, 7961 Citrus Park Town Center Mall, Ta...","7961 Citrus Park Town Center Mall, Tampa"
...,...,...,...,...,...,...
170,0x88c2fb68ca62728b:0x9b38b5a35e75bc94,8xjBQgZsIivU_qfbXYGrZw,Southie Coffee,Southie Coffee,"Southie Coffee, 13045 Park Blvd N, Seminole, F...","13045 Park Blvd N, Seminole"
171,0x88c2c13afd782251:0x816665c854d33199,pdve_-WVRC8V5XWzwHgchw,Steve's Pizzeria,Steve's Pizzeria,"Steve's Pizzeria, 8830 N Himes Ave, Tampa, FL ...","8830 N Himes Ave, Tampa"
172,0x88c2b79fa85011e1:0x19f5c6e1925c64f4,z_bTPCOeRv7IDHSNF0bxzg,Dunkin',Dunkin',"Dunkin', 18003 Highwoods Preserve Pkwy, Tampa,...","18003 Highwoods Preserve Pkwy, Tampa"
173,0x88c2c58a29355281:0xdf75e48d9b8345e3,L16qpZbOaA7bML58m5A0DA,Carlos Sandwich Shop,Carlo's Sandwich Shop,"Carlos Sandwich Shop, 5713 E Columbus Dr, Tamp...","5713 E Columbus Dr, Tampa"


### Para realizar la union de ambos set de datos (google y yelpl) modificaremos su estructura para que los nombres coincidan.

In [9]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'metadata_FL_Google.json'
# Lee el archivo JSON y carga los datos en un DataFrame
df_metadada_google = pd.read_json(ruta_archivo, encoding='utf-8')


# Especifica la ruta del archivo JSON
ruta_archivo = 'yelp_business_fl.json'
# Lee el archivo JSON y carga los datos en un DataFrame
df_metadada_yelp = pd.read_json(ruta_archivo, encoding='utf-8')

In [10]:
# Eliminamos las filas que contengan datos duplicados
df_metadada_google = df_metadada_google.drop_duplicates(subset=["gmap_id"])
# Eiminamos las columnas que no sirvan para el analisis
df_metadada_google = df_metadada_google.drop(['estado'], axis=1)
# Convertimos a numericos los datos de geolocalizacion
df_metadada_google['latitude'] = pd.to_numeric(df_metadada_google['latitude'], errors='coerce')
df_metadada_google['longitude'] = pd.to_numeric(df_metadada_google['longitude'], errors='coerce')
# renombramos las columnas para luego unificar los set de datos
df_metadada_google.columns= ['nombre', 'ubicacion', 'identificador_google', 'latitud', 'longitud', 'categorias', 'avg_rating', 'reviews_count', 'servicios', 'estado']

# Eliminamos las filas que contengas datos dulpicados
df_metadada_yelp = df_metadada_yelp.drop_duplicates(subset=["business_id"])
# Eliminamos las columnas que nos sirvan para el analisis
df_metadada_yelp = df_metadada_yelp.drop(['state'], axis=1)
# Convertimos a numericos los datos de geolocalizacion
df_metadada_yelp['latitude'] = pd.to_numeric(df_metadada_yelp['latitude'], errors='coerce')
df_metadada_yelp['longitude'] = pd.to_numeric(df_metadada_yelp['longitude'], errors='coerce')
# Renombramos las columnas para luego unificar los set de datos
df_metadada_yelp.columns= ['identificador_yelp', 'nombre', 'direccion', 'ciudad', 'latitud', 'longitud', 'avg_rating', 'reviews_count', 'servicios', 'categorias']

In [11]:
# La "ubicacion" de google contiene repetido el nombre del local. eliminaremmos dicho dato de la columna
df_metadada_google['n_ubicacion'] = df_metadada_google.apply(lambda row: row['ubicacion'].replace(row['nombre'] + ', ', ''), axis=1)
# Dividir la columna 'n_ubicacion' en partes usando la coma como separador, obteniendo la direccion, ciudad y estado por separado
df_metadada_google[['direccion', 'ciudad', 'codigo_postal']] = df_metadada_google['n_ubicacion'].str.split(', ', n=2, expand=True)
# Eliminamos las columnas auxiliares y las que no sirven para el analisis.
df_metadada_google = df_metadada_google.drop(['n_ubicacion', 'ubicacion', 'codigo_postal'], axis=1)

In [12]:
df_metadada_google.head(2)          # 15198 filas

,nombre,identificador_google,latitud,longitud,categorias,avg_rating,reviews_count,servicios,estado,direccion,ciudad
0,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,1,"{'Service options': ['Dine-in', 'Delivery'], '...",None,603 Del Prado Blvd S,Cape Coral
1,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[Buffet restaurant],5.0,2,"{'Service options': ['Delivery'], 'Accessibili...",Permanently closed,10525 US-19,Pinellas Park


In [13]:
df_metadada_yelp.head(2)            # 11413 filas

,identificador_yelp,nombre,direccion,ciudad,latitud,longitud,avg_rating,reviews_count,servicios,categorias
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants"
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,53.549633,-113.508780,5.0,20,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad..."


In [14]:
# Concatenamos los set de datos preparados
df_restaurantes = pd.concat([df_metadada_google, df_metadada_yelp], ignore_index=True)

In [15]:
df_restaurantes.head(2)          # 26611 filas

,nombre,identificador_google,latitud,longitud,categorias,avg_rating,reviews_count,servicios,estado,direccion,ciudad,identificador_yelp
0,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,1,"{'Service options': ['Dine-in', 'Delivery'], '...",None,603 Del Prado Blvd S,Cape Coral,NaN
1,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[Buffet restaurant],5.0,2,"{'Service options': ['Delivery'], 'Accessibili...",Permanently closed,10525 US-19,Pinellas Park,NaN


In [16]:
# Teniendo en cuenta los duplicados, detectados mas arriba, los quitamos del dataframe general.
df_restaurantes = df_restaurantes[~df_restaurantes['identificador_google'].isin(df_duplicados['gmap_id'])]

In [17]:
df_restaurantes          # 26436 filas, se eliminaron los 175 duplicados

In [ ]:
df_restaurantes = df_restaurantes.reset_index(drop=False)

In [18]:
# Guardar el DataFrame como un archivo JSON
df_restaurantes.to_json('restaurantes_A.json', orient='records', lines=True)